### Import Libraries

In [1]:
import os
import sys

import json
import numpy as np

In [2]:
NEW_CATEGORIES = ["coral_a", 'coral_b', 'coral_c', 'coral_d']

In [3]:
### Basic Utils

def read_coco_file(coco_file_address):
    open_file = open(coco_file_address)
    coco_file = json.load(open_file)
    return coco_file

def save_coco_file(coco_file, coco_file_address):
    with open(coco_file_address, 'w') as f:
        json.dump(coco_file, f)
    return None

def get_minimum_items_idx(list_arr, n):
    ordered_idx = list(np.argsort(list_arr))[:n]
    ordered_items = list(np.array(list_arr)[ordered_idx])
    return ordered_idx, ordered_items

### Category mapping based functions

def check_for_abcd(coco_file):
    # this has to be made uniform and stricter
    to_check = NEW_CATEGORIES
    categories_list = coco_file["categories"]
    cat_names = [cat['name'].lower() for cat in categories_list]
    for item_to_check in to_check:
        if item_to_check in cat_names:
            return True
    return False

def turn_abcd_lower(coco_file):
    categories_list = coco_file["categories"]
    for idx, cat in enumerate(categories_list):
        categories_list[idx]['name'] = cat["name"].lower()
    return coco_file

def create_new_category_mapping(coco_file):
    categories_list = coco_file["categories"]
    ids = [cat['id'] for cat in categories_list]
    last_id = max(ids)
    coco_file['categories'].extend(get_new_categories(last_id))
    return coco_file

def get_new_categories(last_id):
    new_list = []
    for idx, cat in enumerate(NEW_CATEGORIES):
        new_list.append({"id":last_id + idx + 1, "name" : cat })
    return new_list

def get_id_class_map(coco_file):
    new_dict = {}
    for item in coco_file['categories']:
        new_dict[item['id']] = item['name']
    return new_dict

### Annotation based functions

def get_annotation_center_info(all_bboxs, all_classes, id_class_map):
    annotation_center_arr = []
    for idx, bbox in enumerate(all_bboxs):
        center_of_seg = get_center_of_bbox(bbox)
        annotation_center_arr.append(center_of_seg)
        if id_class_map[all_classes[idx]] == "ref":
            ref_center = center_of_seg
    return annotation_center_arr, ref_center

def get_center_of_bbox(bbox):
    x0, y0, w, h = bbox
    center_x = x0 + w/2
    center_y = y0 + h/2
    return center_x, center_y

def get_distance_arr(center_arr, ref_center):
    distance_arr = []
    for center in center_arr:
        distance = ((center[0] - ref_center[0])**2 + (center[1] - ref_center[1])**2)**(1/2)
        distance_arr.append(distance)
    return distance_arr

def get_image_id_annotation_map(coco_file):
    image_id_dict = {}
    for annotation in coco_file["annotations"]:
        image_id = annotation['image_id']
        if image_id not in image_id_dict.keys():
            image_id_dict[image_id] = []
        image_id_dict[image_id].append(annotation)
    return image_id_dict

def get_new_annotations(image_annotations, id_class_map):
    all_bboxs = [annotation['bbox'] for annotation in image_annotations]
    all_classes = [annotation['category_id'] for annotation in image_annotations]

    annotation_center_arr, ref_center = get_annotation_center_info(all_bboxs, all_classes, id_class_map)
    distance_arr = get_distance_arr(annotation_center_arr, ref_center)
    min_dist_idx, min_dists = get_minimum_items_idx(distance_arr, 5)
    min_dist_idx = min_dist_idx[1:] ### because the first item refers to the reference block distance

    coral_annotation_ids = get_coral_annotation_ids(annotation_center_arr, min_dist_idx, id_class_map)

    for idx, coral_annotation_id in enumerate(coral_annotation_ids):
        if coral_annotation_id!=None:
            image_annotations[min_dist_idx[idx]]['category_id'] = coral_annotation_id

    return image_annotations

def get_coral_annotation_ids(annotation_center_arr, min_dist_idx, id_class_map):
    class_id_map = {j:i for i, j in id_class_map.items()}

    centers = list(np.array(annotation_center_arr)[min_dist_idx])
    all_xs = [center[0] for center in centers]
    all_ys = [center[1] for center in centers]

    ordered_x_idx, min_xs = get_minimum_items_idx(all_xs, 4)
    min_x_idx = ordered_x_idx[:2]
    max_x_idx = ordered_x_idx[2:]

    temp_ys = list(np.array(all_ys)[min_x_idx])
    min_temp_y_idx, _ = get_minimum_items_idx(temp_ys, 2)

    try:
        coral_a_idx = np.array(min_x_idx)[min_temp_y_idx[0]]
    except IndexError:
        coral_a_idx = None
    try:
        coral_c_idx = np.array(min_x_idx)[min_temp_y_idx[1]]
    except IndexError:
        coral_c_idx = None

    temp_ys = list(np.array(all_ys)[max_x_idx])
    min_temp_y_idx, _ = get_minimum_items_idx(temp_ys, 2)

    try:
        coral_b_idx = np.array(max_x_idx)[min_temp_y_idx[0]]
    except IndexError:
        coral_b_idx = None
    try:
        coral_d_idx = np.array(max_x_idx)[min_temp_y_idx[1]]
    except IndexError:
        coral_d_idx = None

    coral_annotations = [None, None, None, None]

    if coral_a_idx!=None:
        coral_annotations[coral_a_idx] = class_id_map['coral_a']
    if coral_b_idx!=None:
        coral_annotations[coral_b_idx] = class_id_map['coral_b']
    if coral_c_idx!=None:
        coral_annotations[coral_c_idx] = class_id_map['coral_c']
    if coral_d_idx!=None:
        coral_annotations[coral_d_idx] = class_id_map['coral_d']

    return coral_annotations

### Main Process

In [4]:
import os
from glob import glob

In [25]:
main_dir_address = "/content/drive/MyDrive/Projects/Coral Microfragmentation/Coral Monitoring/22-23 Season/"

all_date_names = ["22.12.7", "23.2.28", "23.4.9", "23.2.6", "23.1.6", "23.5.5"]
all_table_names = ["table_4","table_5","table_6","table_7","table_8","table_9","table_10"]

# all_date_names = ["23.2.6"]
# all_table_names = ["table_9"]

all_coco_addresses = []
all_coco_save_addresses = []

for date in all_date_names:
    for table in all_table_names:
        folder_with_coco_file = os.path.join(main_dir_address, date, table, 'Output', "Annotation Output")
        potential_coco_files = glob(os.path.join(folder_with_coco_file, "*.json"))
        potential_coco_files_lower = [i.lower() for i in potential_coco_files]
        coco_file_idx = [i for i, j in enumerate(potential_coco_files_lower) if 'coco' in j][0]
        coco_file_address = potential_coco_files[coco_file_idx]
        all_coco_addresses.append(coco_file_address)
        all_coco_save_addresses.append(coco_file_address)

In [26]:
for idx, coco_address in enumerate(all_coco_addresses):
    coco_save_address = all_coco_save_addresses[idx]

    coco_file = read_coco_file(coco_address)

    abcd_present = check_for_abcd(coco_file)
    coco_file = turn_abcd_lower(coco_file)
    if not abcd_present:
        # extend current categories with new categories
        coco_file = create_new_category_mapping(coco_file)

        id_class_map = get_id_class_map(coco_file)
        image_id_annotation_map = get_image_id_annotation_map(coco_file)

        all_new_annotations = []
        for image_id, image_annotations in image_id_annotation_map.items():
            new_annotations = get_new_annotations(image_annotations, id_class_map)
            all_new_annotations.extend(new_annotations)

        all_ids = []
        for annotation in all_new_annotations:
            all_ids.append(annotation['id'])

        new_order = np.argsort(all_ids)
        all_new_annotations = list(np.array(all_new_annotations)[new_order])

        coco_file['annotations'] = all_new_annotations
    save_coco_file(coco_file, coco_save_address)